# Mix datasets

This script mix all our datasets together in a way to have everything we need in only one file.

In [1]:
import pandas as pd
from datetime import datetime

### Import datasets

#### IQ :

In [2]:
dfiq = pd.read_csv("HistoricDataset/iqComplete/completeIQDataset.csv", header=0, delimiter=';')
dfiq['date'] = pd.to_datetime(dfiq['date'],utc=True)
dfiq.head()

,date,IQ
0,2015-01-01 00:00:00+00:00,5.0
1,2015-01-02 00:00:00+00:00,3.0
2,2015-01-03 00:00:00+00:00,3.0
3,2015-01-04 00:00:00+00:00,5.0
4,2015-01-05 00:00:00+00:00,4.0


In [3]:
dfiq.dtypes

date    datetime64[ns, UTC]
IQ                  float64
dtype: object

In [4]:
len(dfiq)

1909

#### Synop : 

In [5]:
dfsynop = pd.read_csv("HistoricDataset/synopComplete/completeSynopDataset.csv", header=0, delimiter=';')
dfsynop['date'] = pd.to_datetime(dfsynop['date'],utc=True)
dfsynop.head()

,date,pressure,wind_direction,wind_force,humidity,temperature
0,1996-01-01 00:00:00+00:00,99380,0,0.0,96,275.95
1,1996-01-01 03:00:00+00:00,99440,150,2.0,97,275.25
2,1996-01-01 06:00:00+00:00,99440,150,2.0,97,275.15
3,1996-01-01 09:00:00+00:00,99470,90,2.0,97,275.35
4,1996-01-01 12:00:00+00:00,99470,70,1.0,97,276.45


In [6]:
dfsynop.dtypes

date              datetime64[ns, UTC]
pressure                        int64
wind_direction                  int64
wind_force                    float64
humidity                        int64
temperature                   float64
dtype: object

In [7]:
len(dfsynop)

70381

#### Pollutants:

In [8]:
dfp = pd.read_csv("HistoricDataset/PollDownload/ArchivePollMELClean.csv", header=0, delimiter=';')
dfp['date'] = pd.to_datetime(dfp['date'],utc=True)
dfp.head()

,date,NO2,O3,PM10
0,2012-01-01 00:00:00+00:00,1.0,3.0,1.0
1,2012-01-02 00:00:00+00:00,3.0,2.0,8.0
2,2012-01-03 00:00:00+00:00,3.0,1.0,8.0
3,2012-01-04 00:00:00+00:00,2.0,3.0,4.0
4,2012-01-05 00:00:00+00:00,1.0,4.0,3.0


In [9]:
dfp.dtypes

date    datetime64[ns, UTC]
NO2                 float64
O3                  float64
PM10                float64
dtype: object

In [10]:
len(dfp)

3026

### Mix datasets

We want to associate an IQ to each line depending on the day.

In [11]:
def getDay(row):
    return(row["date"].year,row["date"].month,row["date"].day)

dfsynop["day"] = dfsynop.apply(lambda row: getDay(row), axis=1)
dfiq["day"] = dfiq.apply(lambda row: getDay(row), axis=1)
dfp["day"] = dfiq.apply(lambda row: getDay(row), axis=1)
dfp = dfp.drop(columns="date")

In [12]:
dfMerged = pd.merge(dfiq, dfsynop, how='inner', on="day")
dfMerged = pd.merge(dfMerged,dfp, how='inner', on="day")
dfMerged.head()

,date_x,IQ,day,date_y,pressure,wind_direction,wind_force,humidity,temperature,NO2,O3,PM10
0,2015-01-01 00:00:00+00:00,5.0,"(2015, 1, 1)",2015-01-01 00:00:00+00:00,103030,170,2.4,100,272.85,1.0,3.0,1.0
1,2015-01-01 00:00:00+00:00,5.0,"(2015, 1, 1)",2015-01-01 03:00:00+00:00,102920,160,3.5,95,272.75,1.0,3.0,1.0
2,2015-01-01 00:00:00+00:00,5.0,"(2015, 1, 1)",2015-01-01 06:00:00+00:00,102990,170,3.4,94,272.65,1.0,3.0,1.0
3,2015-01-01 00:00:00+00:00,5.0,"(2015, 1, 1)",2015-01-01 09:00:00+00:00,103010,190,4.7,85,274.35,1.0,3.0,1.0
4,2015-01-01 00:00:00+00:00,5.0,"(2015, 1, 1)",2015-01-01 12:00:00+00:00,102820,190,6.4,72,277.25,1.0,3.0,1.0


In [13]:
dfMerged = dfMerged.drop(columns=["date_x","day"])
dfMerged = dfMerged.rename(columns={"date_y":"date"})
dfMerged.head()

,IQ,date,pressure,wind_direction,wind_force,humidity,temperature,NO2,O3,PM10
0,5.0,2015-01-01 00:00:00+00:00,103030,170,2.4,100,272.85,1.0,3.0,1.0
1,5.0,2015-01-01 03:00:00+00:00,102920,160,3.5,95,272.75,1.0,3.0,1.0
2,5.0,2015-01-01 06:00:00+00:00,102990,170,3.4,94,272.65,1.0,3.0,1.0
3,5.0,2015-01-01 09:00:00+00:00,103010,190,4.7,85,274.35,1.0,3.0,1.0
4,5.0,2015-01-01 12:00:00+00:00,102820,190,6.4,72,277.25,1.0,3.0,1.0


In [14]:
dfMerged = dfMerged.drop_duplicates()

In [15]:
len(dfMerged)

15058

### Save dataset

In [16]:
dfMerged.to_csv("HistoricDataset/completeMixDataset/completeDatasetPol.csv", index=False,sep=';')